In [7]:
from connectors.maximo_connector import MaximoConnector

# instantiate maximo connector.
maximo_connector = MaximoConnector()

In [8]:
from pydantic import BaseModel, Field
from typing import Dict, Union, Any
from langchain.agents import tool
import ast
from langchain_core.messages import HumanMessage, SystemMessage

class MaximoAgentTools:

    class MaiximoPayloadInput(BaseModel):
        maximo_payload: Union[Dict, str] = Field(description="The payload to be sent to Maximo.")

    @tool(args_schema=MaiximoPayloadInput)
    def perform_maximo_operation(maximo_payload: Union[Dict, str]):
        """
        Perform maximo operation for API requests.
        :param state: The state of the agent containing the user input and states to be updated.
        :return: A dictionary containing the Maximo payload.
        """
        # Check to see the maximo payload returned, and the response type to perform the correct action.
        if isinstance(maximo_payload, str):
            maximo_payload = ast.literal_eval(maximo_payload)

        request_type = maximo_payload.get("request_type")
        params = maximo_payload.get("params")

        if request_type.lower() == 'get':
            result = maximo_connector.get_workorder_details(params)
        elif request_type.lower() == 'post':
            result = maximo_connector.post_workorder_details(params)
        else:
            result = {
                "message": "This query is not related to Maximo."
            }

        return {
            "maximo_agent_response": result
        }
    

    class MaximoPayloadGeneratorInput(BaseModel):
        user_input: str = Field(description="Thing to search for")
        system_prompt: SystemMessage = Field(description="System prompt to use for generating the payload")
        llm: Any = Field(description="LLM to use for generating the payload")


    @tool(args_schema=MaximoPayloadGeneratorInput)
    def generate_maximo_payload(user_input, system_prompt, llm) -> Dict[str, Any]:
        """
        Generates the Maximo payload based on the user input.
        :param state: The state of the agent containing the user input and to be updated.
        :return: A dictionary containing the Maximo payload.
        """
        # Check if the user input is classified as a Maximo operation
        user_input = HumanMessage(
            content=user_input
        )
        messages = [
            system_prompt,
            user_input,
        ]

        response = llm.invoke(messages)

        # validate do a dict.
        maximo_payload = ast.literal_eval(response.content)

        return maximo_payload


In [9]:
from langchain_ibm import ChatWatsonx
import os

from dotenv import load_dotenv
_ = load_dotenv()

model = ChatWatsonx(
    model_id="mistralai/mistral-large",
    apikey=os.environ["IBM_CLOUD_APIKEY"],
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
)

query_maximo_tool = [MaximoAgentTools.generate_maximo_payload, MaximoAgentTools.perform_maximo_operation]


maximo_agent = model.bind_tools(query_maximo_tool)

In [14]:
result1 = maximo_agent.invoke("hello")
result1.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'content': ' Hello! How can I assist you today?',
  'response_metadata': {'token_usage': {'completion_tokens': 10,
    'prompt_tokens': 622,
    'total_tokens': 632},
   'model_name': 'mistralai/mistral-large',
   'system_fingerprint': '',
   'finish_reason': 'stop'},
  'type': 'ai',
  'id': 'chatcmpl-5dee96b99f44af118443b32ab0c33a40',
  'usage_metadata': {'input_tokens': 622,
   'output_tokens': 10,
   'total_tokens': 632},
  'tool_calls': [],
  'invalid_tool_calls': []}}

In [15]:
result2 = maximo_agent.invoke("Get the workorder details for workorder 12345?")
result2.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'content': '',
  'additional_kwargs': {'tool_calls': [{'id': 'mXRZWFKtU',
     'type': 'function',
     'function': {'name': 'generate_maximo_payload',
      'arguments': '{"user_input": "Get the work order details for workorder 12345", "system_prompt": {"content": "You handle user requests by generating a payload for a Maximo provider, based on the user request."}, "llm": "chat-350k"}'}}]},
  'response_metadata': {'token_usage': {'completion_tokens': 83,
    'prompt_tokens': 635,
    'total_tokens': 718},
   'model_name': 'mistralai/mistral-large',
   'system_fingerprint': '',
   'finish_reason': 'tool_calls'},
  'type': 'ai',
  'id': 'chatcmpl-a8b66d3ca5dd36a983e26c11749b8f0b',
  'tool_calls': [{'name': 'generate_maximo_payload',
    'args': {'user_input': 'Get the work order details for workorder 12345',
     'system_prompt': {'content': 'You handle user requests by generating a pa

In [17]:
result1.tool_calls

[]

In [18]:
result2.tool_calls

[{'name': 'generate_maximo_payload',
  'args': {'user_input': 'Get the work order details for workorder 12345',
   'system_prompt': {'content': 'You handle user requests by generating a payload for a Maximo provider, based on the user request.'},
   'llm': 'chat-350k'},
  'id': 'mXRZWFKtU',
  'type': 'tool_call'}]

In [ ]:
for tool_call in result2.tool_calls:
    selected_tool = {}